### Imports y Lectura de Data Frame

Implementacion de una funcion (get_encoding_type) para definir el encoding usado en el Data Frame.  
La funcion fue invocada una única vez para obtener el resultado dado que demora bastante en procesar.   
El resultante es 'utf-8' por lo cual se procedió a definirlo explicitamente en el read_csv.  

Posterior revision de los datos obtenidos, tipos y shape.


In [1]:
import pandas as pd
import numpy as np
import re
import chardet

In [2]:
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [3]:
data_location = "../Data/properati.csv"

In [5]:
# Encoding comentado por tratarse de excesivos registros. Resultado obtenido de la funcion es 'utf-8'
# file_encoding = get_encoding_type(data_location)
# file_encoding

'utf-8'

In [68]:
data = pd.read_csv(data_location, sep=",", encoding='utf-8')
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [11]:
data.shape

(121220, 26)

In [8]:
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

### Analisis informacion de Place

Los campos 'place_with_parent_names', 'place_name', 'country_name' y 'state_name' traen informacion de dónde se ubica la propiedad.  

El valor de nulos es consistente a través de 'place_with_parent_names', 'country_name' y 'state_name'. En la columa 'place_name' se encuentran 23 valores nulos.  
El campo 'country_name' tiene un único valor de territorio por lo que se procede a realizar un drop de ese registro junto con el 'place_with_parent_names' por tratarse de inormación redundante.  
Respecto a los 23 nulos en 'place_name' se cuentan con dos alternativas

    * drop de campos faltantes
    
    * broadcastin de valor de place usando un regex para extraer del 'place_with_parent_names' los valores que ya se tienen. 
    

In [60]:
place_with_parent_nulls = data['place_with_parent_names'].isnull().sum()
place_name_nulls = data['place_name'].isnull().sum()
country_name_nulls = data['country_name'].isnull().sum()
state_name_nulls = data['state_name'].isnull().sum()

print("'Place with Parent' valores nulos: ", place_with_parent_nulls)
print("'Place name' valores nulos: ", place_name_nulls)
print("'Country Name' valores nulos: ", country_name_nulls)
print("'State Name' valores nulos: ", state_name_nulls)

'Place with Parent' valores nulos:  0
'Place name' valores nulos:  23
'Country Name' valores nulos:  0
'State Name' valores nulos:  0


In [64]:
data['country_name'].value_counts()

Argentina    121220
Name: country_name, dtype: int64

In [70]:
data['place_name'].value_counts()

Córdoba                9254
Rosario                8504
Mar del Plata          6534
Tigre                  3324
Nordelta               3315
                       ... 
Santa Ana                 1
Calmayo                   1
Speluzzi                  1
Carmen de Patagones       1
Emilia                    1
Name: place_name, Length: 1060, dtype: int64

In [66]:
data['state_name'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

### Drop de información innecesaria

Se *procede a limpiar aquellos datos que no son de relevancia para el objetivo final.  
Las decisiones se tomaron en base a varios criterios diferentes según cada caso

* **'Unnamed: 0:** Indice redundante que no va a ser utilizado en la organización final.
* **'operation':** Único rsultado para el campo. Al tratarse de todas operaciones de venta, se vuelve redundante.
* **'place_with_parent_names':** Información que esta repetida y se encuentra en otros registros.
* **'country_name':** Único rsultado para el campo. Al tratarse de todas operaciones dentro del territorio argentino, se vuelve redundante.
* **'geonames_id':** Información que no aporta información para el proyecto a largo plazo de estimación de precio.
* **'lat-lon':** Merge de las columnas 'lat' y 'lon'
* **'lat':** Información que no aporta información para el proyecto a largo plazo de estimación de precio.
* **'lon':** Información que no aporta información para el proyecto a largo plazo de estimación de precio.
* **'price_per_m2':** Del análisis realizado se desprende que el campo 'price_per_m2' se compone con la misma formula de 'price_usd_per_m2' pero tomando otra superficie como referencia. Es decir 'price_per_m2 = price/'surface_covered_in_m2' mientras que 'price_usd_per_m2' = price/'surface_total_in_m2' 
* **'expenses':** Excesivos registros nulos (88%) mientras que no aportan información para la estimación.
* **'properati_url':** Información inncesaria.
* **'image_thumbnail':** Información inncesaria.


In [71]:
data['operation'].value_counts()

sell    121220
Name: operation, dtype: int64

In [10]:
data['expenses'].isnull().sum()*100/ len(data)

88.23461475004125

In [5]:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [79]:
data_filtrada = data.drop(columns=(['Unnamed: 0','operation','place_with_parent_names','country_name','geonames_id','lat-lon', 'lat', 'lon','price_per_m2','expenses', 'properati_url','image_thumbnail']))
data_filtrada.head(5)

,property_type,place_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,floor,rooms,description,title
0,PH,Mataderos,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


### Price Analisis
Revisión de campos nulos en columnas de precio.  

Drop price.isnull() - price_aprox_usd.isnull() - price_aprox_local_currency.isnull()  
Los 3 campos cuentan el mismo porcentaje de datos faltantes (16,85%). Resta definir si parte de esos registros faltantes no estan imputados en Comentario o Título.

In [80]:
data_filtrada.price_aprox_local_currency.isnull().sum()*100/ len(data_filtrada)

16.837155584886982

In [81]:
data_filtrada.price_aprox_usd.isnull().sum()*100/ len(data_filtrada)

16.837155584886982

In [82]:
data_filtrada.price.isnull().sum()*100/ len(data_filtrada)

16.837155584886982

### Currency Analisis
Revisión de los valores que trae la serie Currency.  

Se encuentran 2 valores fuera del registro esperado
   * PEN: dos resultados 
   * UYU: un resultado

Drop currency para los casos mencionados arriba por ser outlyers de la consigna de territorio Argentino.

In [83]:
data.currency.value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

In [84]:
data_filtrada.shape

(121220, 14)

In [85]:
data_filtrada.drop(data_filtrada.loc[(data_filtrada['currency']=="PEN")].index, inplace=True)
data_filtrada.drop(data_filtrada.loc[(data_filtrada['currency']=="UYU")].index, inplace=True)

In [86]:
data_filtrada.currency.value_counts()

USD    87587
ARS    13219
Name: currency, dtype: int64

### Rename
Renombramiento de las columnas para tener la información más prolija.

In [87]:
data_filtrada.columns

Index(['property_type', 'place_name', 'state_name', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'floor', 'rooms',
       'description', 'title'],
      dtype='object')

In [90]:
data_filtrada_renamed = data_filtrada.rename(columns = {
    'property_type':'Propiedad', 
    'place_name':'Barrio', 
    'state_name':'Localidad', 
    'price':'Precio', 
    'currency':'Moneda',
    'price_aprox_local_currency':'Precio aprox ARS' ,
    'price_aprox_usd': 'Precio aprox USD', 
    'surface_total_in_m2': 'Superfice total M2',
    'surface_covered_in_m2': 'Superficie cubierta M2', 
    'price_usd_per_m2':'Precio USD por M2', 
    'floor': 'Piso', 
    'rooms': 'Ambientes',
    'description': 'Descripcion', 
    'title' : 'Titulo'},  inplace = False)

data_filtrada_renamed.head(3)

,Propiedad,Barrio,Localidad,Precio,Moneda,Precio aprox ARS,Precio aprox USD,Superfice total M2,Superficie cubierta M2,Precio USD por M2,Piso,Ambientes,Descripcion,Titulo
0,PH,Mataderos,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO


In [92]:
data_filtrada_renamed.columns

Index(['Propiedad', 'Barrio', 'Localidad', 'Precio', 'Moneda',
       'Precio aprox ARS', 'Precio aprox USD', 'Superfice total M2',
       'Superficie cubierta M2', 'Precio USD por M2', 'Piso', 'Ambientes',
       'Descripcion', 'Titulo'],
      dtype='object')

In [76]:
data_filtrada_renamed.Localidad.value_counts()

Capital Federal                 32314
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           680
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

In [103]:
data_filtrada_renamed.dtypes

Propiedad                  object
Barrio                     object
Localidad                  object
Precio                    float64
Moneda                     object
Precio aprox ARS          float64
Precio aprox USD          float64
Superfice total M2        float64
Superficie cubierta M2    float64
Precio USD por M2         float64
Piso                      float64
Ambientes                 float64
Descripcion                object
Titulo                     object
dtype: object

In [100]:
data_filtrada_renamed_org = data_filtrada_renamed.pivot_table(['Precio', 'Moneda',
       'Precio aprox ARS', 'Precio aprox USD', 'Superfice total M2',
       'Superficie cubierta M2', 'Precio USD por M2', 'Piso', 'Ambientes',
       'Descripcion', 'Titulo'], index= ['Localidad','Barrio','Propiedad'])
data_filtrada_renamed_org

Ambientes   Piso  \
Localidad                Barrio                  Propiedad                     
Bs.As. G.B.A. Zona Norte  Country Maschwitz Club house       4.000000   9.00   
                          los alamos             apartment   1.000000    NaN   
                         Acacias Blancas         house       6.200000  42.00   
                         Acassuso                PH          4.500000    NaN   
                                                 apartment   3.105263   4.14   
...                                                               ...    ...   
Tucumán                  Tucumán                 store            NaN    NaN   
                         Villa Belgrano          apartment        NaN    NaN   
                         Yerba Buena             apartment   2.923077    NaN   
                                                 house       4.400000    NaN   
                                                 store            NaN    NaN   

                                                                  Precio  \
Localidad                Barrio                  Propiedad                 
Bs.As. G.B.A. Zona Norte  Country Maschwitz Club house      3.116667e+05   
                          los alamos             apartment  8.160000e+04   
                         Acacias Blancas         house      2.495294e+05   
                         Acassuso                PH         2.387500e+05   
                                                 apartment  2.894698e+05   
...                                                                  ...   
Tucumán                  Tucumán                 store      3.666667e+06   
                         Villa Belgrano          apartment           NaN   
                         Yerba Buena             apartment  9.669562e+05   
                                                 house      4.566842e+06   
                                                 store      1.893694e+06   

                                                            Precio USD por M2  \
Localidad                Barrio                  Propiedad                      
Bs.As. G.B.A. Zona Norte  Country Maschwitz Club house            1369.986720   
                          los alamos             apartment        1600.000000   
                         Acacias Blancas         house            1181.861023   
                         Acassuso                PH               1990.304342   
                                                 apartment        2799.079776   
...                                                                       ...   
Tucumán                  Tucumán                 store             469.374992   
                         Villa Belgrano          apartment                NaN   
                         Yerba Buena             apartment         904.461996   
                                                 house            1061.394102   
                                                 store             694.020321   

                                                            Precio aprox ARS  \
Localidad                Barrio                  Propiedad                     
Bs.As. G.B.A. Zona Norte  Country Maschwitz Club house          5.499202e+06   
                          los alamos             apartment      1.439791e+06   
                         Acacias Blancas         house          4.402822e+06   
                         Acassuso                PH             4.212624e+06   
                                                 apartment      5.107549e+06   
...                                                                      ...   
Tucumán                  Tucumán                 store          3.628418e+06   
                         Villa Belgrano          apartment               NaN   
                         Yerba Buena             apartment      1.623067e+06   
                                                 house          5.628282e+06   
                

In [102]:
data_filtrada_renamed_org.dtypes

Ambientes                 float64
Piso                      float64
Precio                    float64
Precio USD por M2         float64
Precio aprox ARS          float64
Precio aprox USD          float64
Superfice total M2        float64
Superficie cubierta M2    float64
dtype: object